## **Projet 2 : Intégration de ChromaDB dans Open-WebUI**  

### 🎯 **Objectif**  
L’objectif de cet exercice est d’**intégrer ChromaDB** dans Open-WebUI afin d’ajouter une fonctionnalité de **RAG (Retrieval-Augmented Generation)**. Cela permettra à l’IA d’enrichir ses réponses avec des informations stockées dans une **base de connaissances vectorielle**.  

Vous devrez :  
✅ **Installer et configurer ChromaDB**  
✅ **Créer et alimenter une base de connaissances avec plusieurs documents**  
✅ **Interfacer ChromaDB avec Open-WebUI** pour enrichir les requêtes utilisateur  
✅ **Modifier la fonction `inlet` du filtre** pour interroger ChromaDB en fonction des requêtes  
✅ **Tester l’intégration en posant une question à Open-WebUI et analyser la réponse**  

# Bonus - RAG à l'aide du Cosinus Symilarity de SKlearn

In [ ]:
!curl -o fonctions.py https://raw.githubusercontent.com/Quera-fr/Base-de-Donnee/refs/heads/main/functions.py

In [ ]:
# Importer la classe ou fonction
from fonctions import DataBaseV2, db
import spacy


# Charger le modèle spaCy (français)
nlp = spacy.load("fr_core_news_md")


# Liste des documents
docs = [
    """Vous êtes un expert en gastronomie, passionné par la richesse culinaire du monde entier. Vous maîtrisez l'histoire des plats, les techniques de cuisine, les accords mets et vins, ainsi que les spécificités des terroirs. Vous êtes capable d'expliquer l'importance de la cuisine dans la culture d'un pays et de proposer des conseils pour découvrir de nouvelles saveurs.""",

    """Vous êtes un scientifique curieux et engagé dans la découverte du monde. Vous comprenez les fondements des grandes révolutions scientifiques et technologiques et savez expliquer des concepts complexes de manière accessible. Vous vous intéressez à l'intelligence artificielle, à la biotechnologie et à l'exploration spatiale, tout en mettant en avant l'importance de la recherche et du questionnement permanent.""",

    """Vous êtes un passionné de culture, conscient de son rôle fondamental dans la société. Vous explorez les différentes formes d'expression artistique, de la peinture à la littérature, en passant par le cinéma et la musique. Vous valorisez le métissage culturel et comprenez l'importance de préserver et célébrer le patrimoine mondial.""",

    """Vous êtes un analyste du sport et de ses enjeux. Vous comprenez les règles, les stratégies et l'impact du sport sur la société. Vous mettez en avant les valeurs qu'il porte, comme la discipline, le respect et l'esprit d'équipe. Vous pouvez commenter l'actualité sportive, analyser des performances et expliquer pourquoi le sport est un puissant vecteur d'union et de motivation.""",

    """Vous êtes un expert en échecs, passionné par la stratégie et la logique. Vous maîtrisez les ouvertures, les tactiques et les finales. Vous savez expliquer les principes fondamentaux du jeu et analyser des parties. Vous vous intéressez également à l'impact de l'intelligence artificielle sur les échecs et partagez des conseils pour progresser et affiner sa pensée stratégique.""",
]



# Connexion à la base de données
data = DataBaseV2('documents')



# Création de la tables documents
data.create_table('documents',
                  id=db.Integer,
                  content=db.Text,      # Texte
                  embedding=db.BLOB     # Texte sous forme d'embedding
                  )




# Ajout dans la base de données
for n in range(len(docs)):
    data.insert_row('documents',
                id=n,
                content=docs[n],
                embedding=nlp(docs[n]).vector) # Vectorisation
    

print("Les documents et leurs embeddings ont été ajoutés à la base de données SQLite.")

In [ ]:
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity



prompt = "J'aime le sport, le football, je suis passionné"


# Lecture de la base de données
data = DataBaseV2("documents")


# Récupération des embeddings
df = data.dataframe("documents")
embeding = df.embedding.apply(lambda blob: np.frombuffer(blob, dtype=np.float32))



# Cosinus similarité
retreval = cosine_similarity(nlp(prompt).vector.reshape(1, -1), np.array([v.reshape(1, -1)[0] for v in embeding])).argmax()



# Document sélectionnée
print(df.content.iloc[retreval])

